In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.random_graphs import newman_watts_strogatz_graph

In [176]:
data_df = pd.read_csv("data/ml-100k/u.data", sep="\t", header=None, names=["user_id", "item_id", "rating", "timestamp"])
data_df["user_id"] = data_df["user_id"].map({b: a for a, b in enumerate(data_df["user_id"].unique())})
data_df["item_id"] = data_df["item_id"].map({b: a for a, b in enumerate(data_df["item_id"].unique())})
data_df.head()

R = np.zeros((data_df["user_id"].nunique(), data_df["item_id"].nunique()))
for _, row_df in data_df.iterrows():
    user_id = row_df["user_id"]
    item_id = row_df["item_id"]
    rating = row_df["rating"]
    R[user_id, item_id] = rating
    
R = (R > 0).astype(int)
O = R.dot(R.T)
O = (O > 0).astype(int)
np.fill_diagonal(O, 0)

G = nx.Graph()
V = data_df["user_id"].unique()
G.add_nodes_from(V)

E = list(zip(*np.where(O)))
G.add_edges_from(E)

In [187]:
n = 1000 # nr. of nodes
k = 5 # nr. of nearest neighbors
p = 0.1 # rewiring probability
G = newman_watts_strogatz_graph(n, k, p)

In [188]:
N = nx.number_of_nodes(G)
degree_counts = Counter(dict(nx.degree(G)).values())
p = np.zeros(N)
for deg, count in degree_counts.items():
    p[deg] = count / N

In [189]:
degree_counts

Counter({5: 287, 4: 656, 6: 50, 7: 7})

In [353]:
N

1000

In [190]:
def G_prime_0(x):
    n = len(p)
    expectation = 0.0
    for k in range(1, n):
        expectation += k * p[k] * np.power(x, k - 1)
    return expectation

In [191]:
def G_primeprime_0(x):
    n = len(p)
    expectation = 0.0
    for k in range(2, n):
        expectation += k * (k - 1) * p[k] * np.power(x, k - 2)
    return expectation

In [192]:
def G_prime_1(x, alpha=1):
    n = len(p)
    return alpha * (1.0 / G_prime_0(1)) * G_primeprime_0(x)

In [193]:
def G_1(x, alpha=1):
    return alpha * G_prime_0(x) / G_prime_0(1)

In [194]:
z1 = G_prime_0(1)
print("z1: %f" % z1)

z1: 4.408000


In [195]:
z2 = G_primeprime_0(1)
print("z2: %f" % z2)

z2: 15.406000


In [196]:
l = (np.log((N - 1) * (z2 - z1) + np.power(z1, 2)) - np.log(np.power(z1, 2))) / np.log(z2 / z1)
print("l: %f" % l)

l: 5.066100


In [303]:
def G_prime_m(x, m, alpha=1):
    if m > 0 and m <= 1:
        return m * G_prime_0(1)
    elif m < 2 and m > 1:
        return (alpha * z2 / z1) ** (m-1) * z1
    elif m >= 2:
        return G_prime_m(G_1(x, alpha), m-1, alpha) * G_prime_1(1, alpha)

m = 3
z_m = G_prime_m(1, m, alpha=.1)
z_m

0.5384410980036297

In [340]:
def G_prime_m(x, m, alpha=1):
    if m == 1:
        return G_prime_0(1)
    elif m >= 2:
        return G_prime_m(G_1(x, alpha), m-1, alpha) * G_prime_1(1, alpha)

m = 2
alpha = 1
print(np.power(alpha * z2/z1, m-1) * z1)
print(G_prime_m(1, m, alpha))

15.405999999999999
15.405999999999999


In [417]:
alpha = z1 / z2 - np.power(z1, 2) / (z2 * (N-1)) - 1e-9
(np.log((N - 1) * (alpha * z2 - z1) + np.power(z1, 2)) - np.log(np.power(z1, 2))) / np.log(alpha * z2 / z1)

C:\Users\pmuellner\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


nan

In [414]:
z1 / z2 - np.power(z1, 2) / (z2 * (N-1))

0.2848598004729383

In [411]:
def f(l):
    y = 1
    for l_ in range(1, int(l)):
        y += np.power(z2/z1, l_-1) * z1
    y += np.power(z2/z1, l-1) * z1
    return y
f(5.092)

1000.8024685230644

In [413]:
plt.scatter(np.arange(0.1, 6, 0.1), [f(x) for x in np.arange(0.1, 6, 0.1)])
plt.ylabel(r"$1 + \sum z_m$")
plt.xlabel(r"$m$")

Text(0.5, 0, '$m$')

In [409]:
from scipy.optimize import minimize
x0 = 5
result = minimize(f, x0, method="Nelder-Mead", tol=1e-9)
l = result["x"][0]
l

5.09113052272005

In [25]:
%matplotlib qt

In [286]:
x = np.arange(4, 10, .1)
plt.scatter(x, [f(l) for l in x], s=1)
plt.axhline(N, linestyle="dashed", c="gray", label=r"$N$")
plt.ylabel(r"$1 + \sigma z_m$")
plt.xlabel(r"$m$")
plt.legend()

In [287]:
alphas = [0.01, 0.05, 0.1]
for a in alphas:
    M = np.arange(1, 10, 0.05)
    L = [G_prime_m(1, m, alpha=a) for m in M]
    plt.scatter(M, L, s=1, label=r"$\alpha=%.2f$" % a)
plt.ylabel(r"$z_m$")
plt.xlabel(r"$m \in \mathbb{R}$")
plt.legend()
plt.show()

In [209]:
plt.scatter(np.arange(0.1, 8, 0.1), [G_prime_m(1, m) for m in np.arange(0.1, 8, 0.1)], s=1)